# Install necessary libraries

In [33]:
!pip install transformers datasets

# Import necessary libraries

In [51]:
from datasets import load_dataset, concatenate_datasets, Dataset, load_metric
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from sklearn.metrics import precision_score, recall_score, f1_score

# Check if GPU is available

In [50]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Load the IMDB Movie Review dataset

In [35]:
dataset = load_dataset("imdb")

# Load the RoBERTa tokenizer and model

In [36]:
model = RobertaForSequenceClassification.from_pretrained("distilroberta-base", num_labels=2)
tokenizer = RobertaTokenizer.from_pretrained("distilroberta-base")
model.to(device)  # Move the model to the GPU

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Preprocess the data

In [37]:
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True, max_length=512)

encoded_dataset = dataset.map(preprocess_function, batched=True)

 # Split the data into train and test sets

In [38]:
train_dataset = encoded_dataset['train']

# Separate the classes
class_0 = train_dataset.filter(lambda x: x['label'] == 0)
class_1 = train_dataset.filter(lambda x: x['label'] == 1)

# Split the classes separately to maintain balance
train_0, test_0 = train_test_split(class_0, test_size=0.2, random_state=42)
train_1, test_1 = train_test_split(class_1, test_size=0.2, random_state=42)

# Convert lists back to Dataset objects
train_0 = Dataset.from_dict(train_0)
train_1 = Dataset.from_dict(train_1)
test_0 = Dataset.from_dict(test_0)
test_1 = Dataset.from_dict(test_1)

# Combine the split datasets using concatenate_datasets
train_dataset = concatenate_datasets([train_0, train_1])
test_dataset = concatenate_datasets([test_0, test_1])

# After processing the dataset, clear unnecessary variables

In [40]:
del dataset, encoded_dataset
import gc
gc.collect()  # Force garbage collection

126

# Define the metric for evaluation

In [52]:
accuracy_metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    precision = precision_score(labels, predictions, average="binary", zero_division=1)
    recall = recall_score(labels, predictions, average="binary", zero_division=1)
    f1 = f1_score(labels, predictions, average="binary", zero_division=1)
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Define the training arguments

In [53]:
training_args = TrainingArguments(
    output_dir="./results",               # Directory where model predictions and checkpoints will be written
    eval_strategy="epoch",                # Evaluation strategy: perform evaluation at the end of each epoch
    save_strategy="epoch",                # Save strategy: save the model checkpoint at the end of each epoch
    learning_rate=5e-5,                   # Learning rate for the optimizer
    per_device_train_batch_size=8,        # Batch size per GPU/TPU for training
    per_device_eval_batch_size=8,         # Batch size per GPU/TPU for evaluation
    num_train_epochs=3,                   # Number of training epochs
    weight_decay=0.01,                    # Weight decay to apply (if any) for regularization
    save_total_limit=2,                  # Limit the total number of checkpoints to save. Older checkpoints will be deleted
    load_best_model_at_end=True,          # Whether to load the best model (based on the metric specified) at the end of training
    metric_for_best_model="f1",           # Metric to use for selecting the best model. The model with the highest F1 score will be saved
)

# Initialize the Trainer

In [43]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Train the model

In [44]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.290100,0.390854,0.902600,0.953174,0.846800,0.896844
2,0.196300,0.336421,0.926800,0.915498,0.940400,0.927782
3,0.096400,0.357610,0.931200,0.924076,0.939600,0.931773


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.290100,0.390854,0.902600,0.953174,0.846800,0.896844
2,0.196300,0.336421,0.926800,0.915498,0.940400,0.927782
3,0.096400,0.357610,0.931200,0.924076,0.939600,0.931773


TrainOutput(global_step=7500, training_loss=0.2186727294921875, metrics={'train_runtime': 3316.9574, 'train_samples_per_second': 18.089, 'train_steps_per_second': 2.261, 'total_flos': 7948043919360000.0, 'train_loss': 0.2186727294921875, 'epoch': 3.0})

# Evaluate the model

In [45]:
metrics = trainer.evaluate()
print(metrics)

{'eval_loss': 0.3576098084449768, 'eval_accuracy': 0.9312, 'eval_precision': 0.9240755310778914, 'eval_recall': 0.9396, 'eval_f1': 0.931773105910353, 'eval_runtime': 82.3827, 'eval_samples_per_second': 60.692, 'eval_steps_per_second': 7.587, 'epoch': 3.0}


# Save the model and tokenizer

In [46]:
model.save_pretrained("./imdb_finetuned_roberta")
tokenizer.save_pretrained("./imdb_finetuned_roberta")

('./imdb_finetuned_roberta/tokenizer_config.json',
 './imdb_finetuned_roberta/special_tokens_map.json',
 './imdb_finetuned_roberta/vocab.json',
 './imdb_finetuned_roberta/merges.txt',
 './imdb_finetuned_roberta/added_tokens.json')

In [47]:
from collections import Counter

# Checking class distribution in the evaluation set
print(Counter(test_dataset['label']))


Counter({0: 2500, 1: 2500})
